<a href="https://colab.research.google.com/github/peterchang0414/lecun1989-flax/blob/main/lecun_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flax

     |████████████████████████████████| 184 kB 8.4 MB/s 
     |████████████████████████████████| 136 kB 46.5 MB/s 
     |████████████████████████████████| 70 kB 6.2 MB/s 


In [178]:
import jax
import jax.numpy as jnp
import numpy as np

from flax import linen as nn
from torchvision import datasets

data = datasets.MNIST('./data', train=True, download=True)

# Adapted from https://github.com/karpathy/lecun1989-repro/blob/master/prepro.py
def get_datasets():
  train_test = {}
  for split in {'train', 'test'}:
    data = datasets.MNIST('./data', train=split=='train', download=True)
    
    n = 1000 if split == 'train' else 400
    key = jax.random.PRNGKey(0)
    rp = jax.random.permutation(key, len(data))[:n]

    X = jnp.full((n, 16, 16, 1), 0.0, dtype=jnp.float32)
    Y = jnp.full((n, 10), -1.0, dtype=jnp.float32)
    for i, ix in enumerate(rp):
      I, yint = data[int(ix)]
      xi = jnp.array(I, dtype=np.float32) / 127.5 - 1.0
      xi = jax.image.resize(xi, (16, 16), 'bilinear')
      X = X.at[i].set(np.expand_dims(xi, axis=2))
      Y = Y.at[i, yint].set(1.0)
    train_test[split] = (X, Y)
  return train_test

In [179]:
from flax import linen as nn
from flax.training import train_state
from flax.linen.activation import tanh
import optax
from typing import Callable

class Net(nn.Module):
  bias_init: Callable = nn.initializers.zeros

  @nn.compact
  def __call__(self, x):
    # H1 layer
    # For weight initialization, Karpathy used numerator of 2.4 
    # which is very close to sqrt(6) used by he_uniform
    # By default, weight-sharing forces bias-sharing and therefore
    # we add the bias separately.
    bias1 = self.param('bias1', self.bias_init, (8, 8, 12))
    bias2 = self.param('bias2', self.bias_init, (4, 4, 12))
    bias3 = self.param('bias3', self.bias_init, (30,))
    bias4 = self.param('bias4', nn.initializers.constant(-1.0), (10,))
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x = nn.Conv(features=12, kernel_size=(5,5), strides=2, padding='VALID',
                use_bias=False, kernel_init=nn.initializers.he_uniform())(x)
    x = tanh(x + bias1)
    # slice1 = nn.Conv(features=8, kernel_size=(5,5), strides=2, padding=-1.0,
    #                 use_bias=False, kernel_init=nn.initializers.he_uniform())(x[..., 0:8])
    # slice1 = nn.Conv(features=8, kernel_size=(5,5), strides=2, padding=-1.0,
    #                 use_bias=False, kernel_init=nn.initializers.he_uniform())(x[..., 0:8])
    x = jnp.pad(x, [(0,0),(2,2),(2,2),(0,0)], constant_values=-1.0)
    x = nn.Conv(features=12, kernel_size=(5,5), strides=2, padding='VALID',
                use_bias=False, kernel_init=nn.initializers.he_uniform())(x)
    x = tanh(x + bias2)
    x = x.reshape((x.shape[0], -1))
    x = nn.Dense(features=30, use_bias=False)(x)
    x = tanh(x + bias3)
    x = nn.Dense(features=10, use_bias=False)(x)
    x = tanh(x + bias4)

    return x


In [180]:
@jax.jit
def loss_fn(params, X, Y):
  Yhat = Net().apply({'params': params}, X)
  loss = jnp.mean((Yhat - Y)**2)
  err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
  return loss, err

In [181]:
def eval_split(data, split, params):
  X, Y = data[split]
  loss, err = loss_fn(params, X, Y)
  print(f"eval: split {split:5s}. loss {loss:e}. error {err*100:.2f}%. misses: {int(err*Y.shape[0])}")

In [182]:
from jax import value_and_grad
import optax
from flax.training import train_state

def create_train_state(key, lr, X):
  model = Net()
  params = model.init(key, X)['params']
  sgd_opt = optax.sgd(lr)
  return train_state.TrainState.create(apply_fn=cnn.apply, params=params, tx=sgd_opt)

@jax.jit
def train_step(state, X, Y):
  def loss_fn(params):
    Yhat = Net().apply({'params': params}, X)
    loss = jnp.mean((Yhat - Y)**2)
    err = jnp.mean(jnp.argmax(Y, -1) != jnp.argmax(Yhat, -1)).astype(float)
    return loss, err
  (_, Yhats), grads = jax.value_and_grad(loss_fn, has_aux=True)(state.params)
  state = state.apply_gradients(grads=grads)
  return state

def train_one_epoch(state, X, Y):
  for step_num in range(X.shape[0]):
    x, y = np.expand_dims(X[step_num], 0), np.expand_dims(Y[step_num], 0)
    state = train_step(state, x, y)
  return state

def train(key, data, epochs, lr):
  Xtr, Ytr = data['train']
  Xte, Yte = data['test']
  train_state = create_train_state(key, lr, Xtr)
  for epoch in range(epochs):
    print(f"epoch {epoch}...\n")
    train_state = train_one_epoch(train_state, Xtr, Ytr)
    for split in ['train', 'test']:
      eval_split(data, split, train_state.params)


In [183]:
key = jax.random.PRNGKey(0)

train(key, get_datasets(), 10, 0.03)

epoch 0...

eval: split train. loss 1.549302e-01. error 18.30%. misses: 183
eval: split test . loss 1.535312e-01. error 17.75%. misses: 71
epoch 1...

eval: split train. loss 9.297146e-02. error 10.80%. misses: 108
eval: split test . loss 9.958083e-02. error 11.75%. misses: 47
epoch 2...

eval: split train. loss 6.479292e-02. error 6.90%. misses: 69
eval: split test . loss 7.811233e-02. error 9.75%. misses: 39
epoch 3...

eval: split train. loss 5.236620e-02. error 6.10%. misses: 61
eval: split test . loss 7.001271e-02. error 9.50%. misses: 38
epoch 4...

eval: split train. loss 4.351507e-02. error 5.30%. misses: 53
eval: split test . loss 6.512135e-02. error 9.25%. misses: 37
epoch 5...

eval: split train. loss 3.656428e-02. error 3.80%. misses: 38
eval: split test . loss 6.199556e-02. error 9.25%. misses: 37
epoch 6...

eval: split train. loss 3.152552e-02. error 3.30%. misses: 33
eval: split test . loss 6.014356e-02. error 9.50%. misses: 38
epoch 7...

eval: split train. loss 2.7930

In [103]:
train_test = get_datasets()

In [104]:
train = train_test['train'][0]

In [105]:
train.shape

(1000, 16, 16, 1)

In [106]:
key = jax.random.PRNGKey(0)
cnn = Net()
param = cnn.init(key, train)['params']

1:  (1000, 16, 16, 1)
2:  (1000, 20, 20, 1)
3:  (1000, 8, 8, 12)
4:  (1000, 8, 8, 12)
5:  (1000, 12, 12, 12)
6:  (1000, 4, 4, 12)
7:  (1000, 4, 4, 12)
8:  (1000, 30)
9:  (1000, 10)


In [107]:
print(jax.tree_map(lambda x: x.shape, param))

FrozenDict({
    Conv_0: {
        kernel: (5, 5, 1, 12),
    },
    Conv_1: {
        kernel: (5, 5, 12, 12),
    },
    Dense_0: {
        kernel: (192, 30),
    },
    Dense_1: {
        kernel: (30, 10),
    },
    bias1: (8, 8, 12),
    bias2: (4, 4, 12),
    bias3: (30,),
    bias4: (10,),
})


In [113]:
eval_split(train_test, 'train', param)

1:  (1000, 16, 16, 1)
2:  (1000, 20, 20, 1)
3:  (1000, 8, 8, 12)
4:  (1000, 8, 8, 12)
5:  (1000, 12, 12, 12)
6:  (1000, 4, 4, 12)
7:  (1000, 4, 4, 12)
8:  (1000, 30)
9:  (1000, 10)
eval: split train. loss 4.543942e-01. error 87.60%. misses: 876
